<a href="https://colab.research.google.com/github/mistryvivek/flowers-nn-int2/blob/main/PyTorch_reattempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [196]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [197]:
# Download training data from open datasets.
training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform= transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]),
)

In [198]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [208]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(400, 6)
        self.fc2 = nn.Linear(6, 1)
        self.fc3 = nn.Linear(1, 102)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [200]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [202]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(training_data, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, torch.Tensor([labels]))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')

print('Finished Training')

Streaming output truncated to the last 5000 lines.
[46,   102] loss: 114.021
[46,   103] loss: 114.875
[46,   104] loss: 115.729
[46,   105] loss: 116.583
[46,   106] loss: 117.436
[46,   107] loss: 118.289
[46,   108] loss: 119.141
[46,   109] loss: 119.993
[46,   110] loss: 120.845
[46,   111] loss: 121.656
[46,   112] loss: 122.466
[46,   113] loss: 123.276
[46,   114] loss: 124.086
[46,   115] loss: 124.895
[46,   116] loss: 125.704
[46,   117] loss: 126.513
[46,   118] loss: 127.321
[46,   119] loss: 128.129
[46,   120] loss: 128.937
[46,   121] loss: 129.705
[46,   122] loss: 130.472
[46,   123] loss: 131.240
[46,   124] loss: 132.006
[46,   125] loss: 132.773
[46,   126] loss: 133.539
[46,   127] loss: 134.305
[46,   128] loss: 135.071
[46,   129] loss: 135.836
[46,   130] loss: 136.601
[46,   131] loss: 137.327
[46,   132] loss: 138.053
[46,   133] loss: 138.778
[46,   134] loss: 139.503
[46,   135] loss: 140.228
[46,   136] loss: 140.953
[46,   137] loss: 141.677
[46,   138] l

In [209]:
    num_correct = 0
    num_samples = 0
    net.eval()
    
    with torch.no_grad():
        for x, y in test_dataloader:
            scores = net(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
    net.train()

Got 34 / 6149 with accuracy 0.55


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=1, bias=True)
  (fc3): Linear(in_features=1, out_features=102, bias=True)
)